# Results on the News datasets

In [ ]:
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', None)

In [ ]:
# Load dataframes
df_mvae = pd.read_csv('../outputs/sweep_outputs/news.csv')
df_or = pd.read_csv('../outputs/outcome_regression_outputs/news.csv')
df_match = pd.read_csv('../outputs/matching_benchmark_outputs/news.csv')
df_pvae = pd.read_csv('../outputs/matching_benchmark_outputs/plainvae_news.csv')

# Put columns from all matrices in the right format and select the appropriate ones
df_mvae = df_mvae.rename(columns={'{}_matching_metric/{}_{}'.format(dataset, model, metric): '{}_{}_{}'.format(model, metric, dataset_2) for (dataset, dataset_2) in zip(['train','test'],['in','out']) for model in ['z','balancing_1','balancing_2','balancing_3','balancing_4','propensity'] for metric in ['error_att','linear_mmd_att']})
df_mvae = df_mvae[[col for col in df_mvae.columns if col[-3:] in ['_in','out'] and ('error_att' in col or 'linear_mmd_att' in col)]]
df_or = df_or[[col for col in df_or.columns if 'error_att' in col]]
df_match = df_match[[col for col in df_match.columns if ('error_att' in col or 'linear_mmd_att' in col) and 'ipw' not in col and 'ground truth' not in col]]
df_pvae = df_pvae[[col for col in df_pvae.columns if ('error_att' in col or 'linear_mmd_att' in col) and 'ground truth' not in col and 'no matching' not in col]]

In [ ]:
# Proper naming of models
atlas = {
    
    'z': 'Latent space',
    'balancing_1': 'Balancing 1',
    'balancing_2': 'Balancing 2',
    'balancing_3': 'Balancing 3',
    'balancing_4': 'Balancing 4',
    'propensity': 'PS wrt Z',
    
    'TNet': 'TNet',
    'SNet1': 'SNet-1',
    'SNet2': 'SNet-2',
    'SNet3': 'SNet-3',
    'SNet': 'SNet',
    'PseudoOutcomeNet_RA': 'RA-Learner + TNet',
    'PseudoOutcomeNet_RA_S2': 'RA-Learner + SNet-2 NAP',
    'PseudoOutcomeNet_DR': 'DR-Learner + TNet',
    'PseudoOutcomeNet_PW': 'PW-Learner + TNet NAP',
    
    'random': 'Random',
    'x_eucl': 'X',
    'propensity_x': 'LogReg PS',
    'x_pca_eucl': 'PCA',
    'propensity_x_pca': 'PCA + LogReg PS',
    'plain_vae': 'Plain VAE',
    'propensity_plain_vae': 'Plain VAE + LogReg PS',
    'no matching': 'No Matching'
    
}

Note : all results appear in the paper's LaTeX format, with ATT error on in-sample and hold-out data then imbalance on in-sample and hold-out data, in this order.

In [ ]:
# Results on outcome regression methods
# Note: "NAP" means "Not In Paper", showing methods that were not put in the paper due to a lack of space
rename = (lambda s: s.replace('_in','').replace('_out','').replace('_error_att','').replace('_linear_mmd_att',''))

show = (df_or[[col for col in df_or.columns if 'error_att_in' in col]].mean().round(3).astype(str).rename(rename)\
+ r'$\pm$' + (df_or[[col for col in df_or.columns if 'error_att_in' in col]].std() / np.sqrt(50)).round(3).astype(str).rename(rename)\
+ ' & ' + df_or[[col for col in df_or.columns if 'error_att_out' in col]].mean().round(3).astype(str).rename(rename)\
+ r'$\pm$' + (df_or[[col for col in df_or.columns if 'error_att_out' in col]].std() / np.sqrt(50)).round(3).astype(str).rename(rename))
show = show.rename(atlas)
show = show.loc[show.index.intersection(atlas.values())]
show = (show.index + ' & ' + show + r' & NA & NA \\')


for el in show.values:
    print(el)

In [ ]:
# Results on matchVAE scores
rename = (lambda s: s.replace('_in','').replace('_out','').replace('_error_att','').replace('_linear_mmd_att',''))

show = (df_mvae[[col for col in df_mvae.columns if 'error_att_in' in col]].mean().round(3).astype(str).rename(rename)\
+ r'$\pm$' + (df_mvae[[col for col in df_mvae.columns if 'error_att_in' in col]].std() / np.sqrt(500)).round(3).astype(str).rename(rename)\
+ ' & ' + df_mvae[[col for col in df_mvae.columns if 'error_att_out' in col]].mean().round(3).astype(str).rename(rename)\
+ r'$\pm$' + (df_mvae[[col for col in df_mvae.columns if 'error_att_out' in col]].std() / np.sqrt(500)).round(3).astype(str).rename(rename)\
+ ' & ' + df_mvae[[col for col in df_mvae.columns if 'linear_mmd_att_in' in col]].mean().round(3).astype(str).rename(rename)\
+ r'$\pm$' + (df_mvae[[col for col in df_mvae.columns if 'linear_mmd_att_in' in col]].std() / np.sqrt(500)).round(3).astype(str).rename(rename)\
+ ' & ' + df_mvae[[col for col in df_mvae.columns if 'linear_mmd_att_out' in col]].mean().round(3).astype(str).rename(rename)\
+ r'$\pm$' + (df_mvae[[col for col in df_mvae.columns if 'linear_mmd_att_out' in col]].std() / np.sqrt(500)).round(3).astype(str).rename(rename)\
       )

show = show.rename(atlas)
show = show.loc[show.index.intersection(atlas.values())]
show = (show.index + ' & ' + show + r' \\')


for el in show.values:
    print(el)

In [ ]:
# Results on matching bechmark methods, except those relying on plain VAE
rename = (lambda s: s.replace('_in','').replace('_out','').replace('_error_att','').replace('_linear_mmd_att',''))
df_match['no matching_error_att_in'] = np.nan
df_match['no matching_error_att_out'] = np.nan

show = (df_match[[col for col in df_match.columns if 'error_att_in' in col]].mean().round(3).astype(str).rename(rename)\
+ r'$\pm$' + (df_match[[col for col in df_match.columns if 'error_att_in' in col]].std() / np.sqrt(50)).round(3).astype(str).rename(rename)\
+ ' & ' + df_match[[col for col in df_match.columns if 'error_att_out' in col]].mean().round(3).astype(str).rename(rename)\
+ r'$\pm$' + (df_match[[col for col in df_match.columns if 'error_att_out' in col]].std() / np.sqrt(50)).round(3).astype(str).rename(rename)\
+ ' & ' + df_match[[col for col in df_match.columns if 'linear_mmd_att_in' in col]].mean().round(3).astype(str).rename(rename)\
+ r'$\pm$' + (df_match[[col for col in df_match.columns if 'linear_mmd_att_in' in col]].std() / np.sqrt(50)).round(3).astype(str).rename(rename)\
+ ' & ' + df_match[[col for col in df_match.columns if 'linear_mmd_att_out' in col]].mean().round(3).astype(str).rename(rename)\
+ r'$\pm$' + (df_match[[col for col in df_match.columns if 'linear_mmd_att_out' in col]].std() / np.sqrt(50)).round(3).astype(str).rename(rename)\
       )

show = show.rename(atlas)
show = show.loc[show.index.intersection(atlas.values())]
show = (show.index + ' & ' + show + r' \\').str.replace('nan$\\pm$nan','NA', regex=False)

for el in show.values:
    print(el)

In [ ]:
# Results on plain VAE scores
rename = (lambda s: s.replace('_in','').replace('_out','').replace('_error_att','').replace('_linear_mmd_att',''))

show = (df_pvae[[col for col in df_pvae.columns if 'error_att_in' in col]].mean().round(3).astype(str).rename(rename)\
+ r'$\pm$' + (df_pvae[[col for col in df_pvae.columns if 'error_att_in' in col]].std() / np.sqrt(500)).round(3).astype(str).rename(rename)\
+ ' & ' + df_pvae[[col for col in df_pvae.columns if 'error_att_out' in col]].mean().round(3).astype(str).rename(rename)\
+ r'$\pm$' + (df_pvae[[col for col in df_pvae.columns if 'error_att_out' in col]].std() / np.sqrt(500)).round(3).astype(str).rename(rename)\
+ ' & ' + df_pvae[[col for col in df_pvae.columns if 'linear_mmd_att_in' in col]].mean().round(3).astype(str).rename(rename)\
+ r'$\pm$' + (df_pvae[[col for col in df_pvae.columns if 'linear_mmd_att_in' in col]].std() / np.sqrt(500)).round(3).astype(str).rename(rename)\
+ ' & ' + df_pvae[[col for col in df_pvae.columns if 'linear_mmd_att_out' in col]].mean().round(3).astype(str).rename(rename)\
+ r'$\pm$' + (df_pvae[[col for col in df_pvae.columns if 'linear_mmd_att_out' in col]].std() / np.sqrt(500)).round(3).astype(str).rename(rename)\
       )

show = show.rename(atlas)
show = show.loc[show.index.intersection(atlas.values())]
show = (show.index + ' & ' + show + r' \\')


for el in show.values:
    print(el)

In [ ]:
# Boxplots on MatchVAE vs. outcome regression methods

import pandas
import matplotlib.pyplot as plt
import seaborn as sns

# Boxplots for in-sample and out_sample
df_all = pd.concat([df_mvae[[col for col in df_mvae.columns if 'linear_mmd_att' not in col]], df_or], axis='columns')
results = df_all
results['experiment'] = df_all.index

# do wrangling to required format
results = results.melt(id_vars=['experiment'], var_name='model', value_name='error_att')

results.loc[results['model'].str.contains("_in"), 'dataset'] = 'In-Sample'
results.loc[results['model'].str.contains("_out"), 'dataset'] = 'Hold-out'
# remove _in and _out in name
results.loc[results['model'].str.contains("_in"), 'model'] = results.loc[results['model'].str.contains("_in"), 'model'].str.slice(stop=-3)
results.loc[results['model'].str.contains("_out"), 'model'] = results.loc[results['model'].str.contains("_out"), 'model'].str.slice(stop=-4)

results = results.rename(columns={'error_att': 'ATT Error', 'linear_mmd_att': 'Imbalance', 'model': 'Model', 'dataset': 'Dataset'})
results['Model'] = results['Model'].str.replace('_error_att', '').apply(lambda s: atlas[s])
results = results[~results['Model'].str.contains(' NAP')]

results = results.rename(columns={'error_att': 'ATT Error', 'model': 'Model', 'dataset': 'Dataset'})
results.index = results.Model
results = results.loc[pd.Index(atlas.values()).intersection(results.index)]

plt.rc('figure', figsize=(10,15))
fig, axs = plt.subplots(2,2)
for i,showfliers in enumerate([False,True]):
    for j,res in enumerate([results[results.Dataset == 'In-Sample'], results[results.Dataset == 'Hold-out']]):
        sns.boxplot(x='Model', y='ATT Error', data=res, showfliers=showfliers, ax=axs[i,j], color='grey')#, hue='Dataset')
        plt.setp(axs[i,j])
        if i==0:
            axs[i,j].set_xlabel(None)
        if j>0:
            axs[i,j].set_ylabel(None)
        axs[i,j].set_xticklabels(axs[i,j].get_xticklabels(),rotation=90, size=10)
fig.tight_layout()
plt.show()

In [ ]:
# Boxplots on matchings

import pandas
import matplotlib.pyplot as plt
import seaborn as sns

df_all = pd.concat([df_mvae, df_match, df_pvae], axis='columns')
results = df_all
results['experiment'] = df_all.index

# do wrangling to required format
results_error_att = results[[col for col in results.columns if 'error_att' in col or col == 'experiment']].melt(id_vars=['experiment'], var_name='model', value_name='error_att')
results_linear_mmd = results[[col for col in results.columns if 'linear_mmd_att' in col or col == 'experiment']].melt(id_vars=['experiment'], var_name='model', value_name='linear_mmd_att')

results_error_att.loc[results_error_att['model'].str.contains("_in"), 'dataset'] = 'In-Sample'
results_error_att.loc[results_error_att['model'].str.contains("_out"), 'dataset'] = 'Hold-out'

results_linear_mmd.loc[results_linear_mmd['model'].str.contains("_in"), 'dataset'] = 'In-Sample'
results_linear_mmd.loc[results_linear_mmd['model'].str.contains("_out"), 'dataset'] = 'Hold-out'

# remove _in and _out in name
results_error_att.loc[results_error_att['model'].str.contains("_in"), 'model'] = results_error_att.loc[results_error_att['model'].str.contains("_in"), 'model'].str.slice(stop=-3)
results_error_att.loc[results_error_att['model'].str.contains("_out"), 'model'] = results_error_att.loc[results_error_att['model'].str.contains("_out"), 'model'].str.slice(stop=-4)

results_linear_mmd.loc[results_linear_mmd['model'].str.contains("_in"), 'model'] = results_linear_mmd.loc[results_linear_mmd['model'].str.contains("_in"), 'model'].str.slice(stop=-3)
results_linear_mmd.loc[results_linear_mmd['model'].str.contains("_out"), 'model'] = results_linear_mmd.loc[results_linear_mmd['model'].str.contains("_out"), 'model'].str.slice(stop=-4)



results_error_att = results_error_att.rename(columns={'error_att': 'ATT Error', 'linear_mmd_att': 'Imbalance', 'model': 'Model', 'dataset': 'Dataset'})
results_error_att['Model'] = results_error_att['Model'].str.replace('_error_att', '').apply(lambda s: atlas[s])
results_linear_mmd = results_linear_mmd.rename(columns={'_error_att': 'ATT Error', 'linear_mmd_att': 'Imbalance', 'model': 'Model', 'dataset': 'Dataset'})
results_linear_mmd['Model'] = results_linear_mmd['Model'].str.replace('_linear_mmd_att', '').apply(lambda s: atlas[s])
results_error_att.index = results_error_att.Model
results_error_att = results_error_att.loc[pd.Index(atlas.values()).intersection(results_error_att.index)]
results_linear_mmd.index = results_linear_mmd.Model
results_linear_mmd = results_linear_mmd.loc[pd.Index(atlas.values()).intersection(results_linear_mmd.index)]

for results in [results_error_att,results_linear_mmd]:
    plt.rc('figure', figsize=(10,15))
    fig, axs = plt.subplots(2,2)
    for i,showfliers in enumerate([False,True]):
        for j,res in enumerate([results[results.Dataset == 'In-Sample'], results[results.Dataset == 'Hold-out']]):
            sns.boxplot(x='Model', y='ATT Error' if 'ATT Error' in res.columns else 'Imbalance', data=res, showfliers=showfliers, ax=axs[i,j], color='grey')#, hue='Dataset')
            plt.setp(axs[i,j])
            if i==0:
                axs[i,j].set_xlabel(None)
            if j>0:
                axs[i,j].set_ylabel(None)
            axs[i,j].legend(loc='upper left')
            axs[i,j].set_xticklabels(axs[i,j].get_xticklabels(),rotation=90, size=10)
    fig.tight_layout()
    fig.show()